In [1]:
#dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date


In [2]:
#import data
measurements = pd.read_csv('clean_hawaii_measurements.csv')
stations = pd.read_csv('clean_hawaii_stations.csv')
measurements['date'] = pd.to_datetime(measurements['date'], format='%Y-%m-%d')
measurements.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.00,73


In [3]:
#convert to dict
measurements_data = measurements.to_dict(orient='records')
measurements_data[0]

{'Unnamed: 0': 0,
 'date': Timestamp('2010-01-01 00:00:00'),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [4]:
#create engine
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

conn = engine.connect() 

In [5]:
#create classes
Base = declarative_base()


class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
    
Base.metadata.create_all(engine)

In [6]:
#add tables
metadata = MetaData(bind=engine)
metadata.reflect()

measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)
#conn.execute(measurement_table.delete())

conn.execute(measurement_table.insert(), measurements_data)


In [7]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'measurements', 'station']

In [9]:
#test insert
conn.execute('select * from measurements limit 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [10]:
#insert station table
station_data = stations.to_dict(orient='records')

station_data[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [11]:
station_table = sqlalchemy.Table('station', metadata, autoload=True)

conn.execute(station_table.delete())

conn.execute(station_table.insert(), station_data)

conn.execute('select * from station limit 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]